In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, confusion_matrix,accuracy_score,roc_auc_score,recall_score

In [55]:
df=pd.read_csv("train.csv",parse_dates=[12,15])

df["Delay"]=df["placement date"]-df["delivery date"]
df["Delay"]=df["Delay"]/ np.timedelta64(1,"D")

df["delivery type"]=df["delivery type"].fillna("Same Day")

# del df["RID"]

del df["delivery date"]
del df["placement date"]
del df["departure city"]
del df["departure state"]
# del df["address code"]

d={"Home Office":3,"Corporate":2,"Consumer":1}
df["segment"]=df["segment"].map(d)
d={"Central":1,"East":2,"West":3,"South":4}
df["location"]=df["location"].map(d)


d={"kariox":2,"qexty":1,"fynota":3}
df["class"]=df["class"].map(d)


h={}
n=1
for i in df.groupby("sub-class")["profit"].sum().sort_values().index:
    h[i]=n
    n+=1
df["sub-class"]=df["sub-class"].map(h)    


train_df=pd.get_dummies(df,columns=["delivery type"],drop_first=True)

y_train=df["profit"]
x_train=train_df.drop("profit",axis=1)

test=pd.read_csv("test.csv",parse_dates=[11,14])

del x_train["id"]

x_test=test.copy()

del x_test["id"]

x_test["Delay"]=x_test["placement date"]-x_test["delivery date"]
x_test["Delay"]=x_test["Delay"]/ np.timedelta64(1,"D")

del x_test["delivery date"]
del x_test["placement date"]
del x_test["departure city"]
del x_test["departure state"]

# del x_test["address code"]
d={"Home Office":3,"Corporate":2,"Consumer":1}
x_test["segment"]=x_test["segment"].map(d)
d={"Central":1,"East":2,"West":3,"South":4}
x_test["location"]=x_test["location"].map(d)


d={"kariox":2,"qexty":1,"fynota":3}
x_test["class"]=x_test["class"].map(d)



x_test["sub-class"]=x_test["sub-class"].map(h)    


x_test=pd.get_dummies(x_test,columns=["delivery type"],drop_first=True)

# del x_test["RID"]

import xgboost as xgb

xgb_clf = xgb.XGBRegressor(n_estimators=1000,learning_rate=0.2,max_depth=3,gamma=0.2,subsample=0.8,reg_alpha=0.05,colsample_bytree=0.9)

xgb_clf.fit(x_train, y_train)


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9, gamma=0.2, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.2, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0.05, reg_lambda=1, scale_pos_weight=1, subsample=0.8,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [56]:
from sklearn.metrics import r2_score, confusion_matrix,accuracy_score,roc_auc_score,recall_score
r2_score(y_train,xgb_clf.predict(x_train) )


0.9974339036841527

In [58]:
x_train.Delay.unique()

array([  0., -31., -28., -30., -29.])

In [11]:
x_test

,discount,price,no of items,location,class,segment,sub-class,RID,address code,Delay,delivery type_Same Day,delivery type_Second Class,delivery type_Standard Class
0,0.0,24.200,5,3,2,1,6,2408,94122,0.0,0,0,1
1,0.2,359.976,3,3,3,1,16,2409,94122,0.0,0,0,1
2,0.0,3.520,2,2,2,1,6,5425,6708,0.0,0,0,1
3,0.2,11.520,5,1,2,1,5,7408,60653,0.0,0,0,0
4,0.0,242.940,3,3,2,3,11,733,98115,0.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3289,0.0,31.920,4,2,2,1,11,1171,10035,0.0,0,1,0
3290,0.0,19.440,3,2,2,1,13,1170,10035,0.0,0,1,0
3291,0.0,272.940,3,2,2,1,11,1169,10035,0.0,0,1,0
3292,0.2,113.568,2,3,1,1,12,414,94110,0.0,0,0,1


In [44]:
final=pd.DataFrame(test["id"])

final["profit"]=xgb_clf.predict(x_test)



In [45]:
final

,id,profit
0,6701,2.013955
1,6702,33.411427
2,6703,-0.379312
3,6704,8.750709
4,6705,18.348099
...,...,...
3289,9990,-14.167995
3290,9991,9.901832
3291,9992,24.464622
3292,9993,4.012878


In [14]:
final.to_csv('submission3.csv', index=False)

# Attempt

In [33]:
df=pd.read_csv("train.csv",parse_dates=[12,15])

df["Delay"]=df["placement date"]-df["delivery date"]
df["Delay"]=df["Delay"]/ np.timedelta64(1,"D")

del df["RID"]

del df["delivery date"]
del df["placement date"]
del df["departure city"]
del df["departure state"]

train_df=pd.get_dummies(df,columns=["location","class","segment","sub-class","delivery type"],drop_first=True)

y=df["profit"]
x=train_df.drop("profit",axis=1)


del x["id"]


In [34]:
x

,discount,price,no of items,address code,Delay,location_East,location_South,location_West,class_kariox,class_qexty,...,sub-class_phone,sub-class_shorts,sub-class_sweets,sub-class_tablet,sub-class_television,sub-class_tshirts,sub-class_watch,delivery type_Same Day,delivery type_Second Class,delivery type_Standard Class
0,0.2,16.448,2,77095,0.0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,1
1,0.0,29.700,5,48185,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,0.0,14.730,3,48185,0.0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1
3,0.0,43.920,3,48185,0.0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
4,0.0,66.580,2,48185,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6695,0.0,542.940,3,5408,0.0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
6696,0.2,276.784,2,95123,0.0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,1
6697,0.0,2.040,1,5408,0.0,1,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1
6698,0.1,207.846,3,10024,0.0,1,0,0,0,1,...,0,0,0,0,0,1,0,0,1,0


In [35]:
y

0         5.5512
1        13.3650
2         4.8609
3        12.7368
4        15.9792
          ...   
6695    152.0232
6696     89.9548
6697      0.9588
6698      2.3094
6699     10.0485
Name: profit, Length: 6700, dtype: float64

In [36]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import normalize

In [37]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [38]:
scalar=preprocessing.StandardScaler()
x_train_s=scalar.fit_transform(x_train)
x_test_s=scalar.fit_transform(x_test)

In [39]:
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(x_train)
x_train_n = pd.DataFrame(np_scaled)

min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(x_test)
x_test_n = pd.DataFrame(np_scaled)

In [42]:
x_train_n

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,0.00,0.000252,0.000000,0.421123,1.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.25,0.014896,0.153846,0.753514,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.25,0.000454,0.307692,0.184142,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.00,0.002712,0.538462,0.926074,1.0,0.0,0.0,1.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.00,0.001085,0.307692,0.987808,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5355,0.00,0.007217,0.153846,0.203580,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5356,0.00,0.011018,0.384615,0.905670,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5357,0.25,0.000693,0.307692,0.184142,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
5358,0.25,0.000280,0.076923,0.905843,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
import xgboost as xgb

xgb_clf = xgb.XGBRegressor(n_estimators=1000,learning_rate=0.2,max_depth=3,gamma=0.2,subsample=0.8,reg_alpha=0.05,colsample_bytree=0.9)

xgb_clf.fit(x_train, y_train)

r2_score(y_train,xgb_clf.predict(x_train) )

0.9982164188707033

In [44]:
r2_score(y_test,xgb_clf.predict(x_test) )

0.6833413840287805

In [45]:
xgb_clf = xgb.XGBRegressor(n_estimators=500,learning_rate=0.2,max_depth=3,gamma=0.2,subsample=0.8,reg_alpha=0.05,colsample_bytree=0.9)

xgb_clf.fit(x_train_s, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9, gamma=0.2, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.2, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=500, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0.05, reg_lambda=1, scale_pos_weight=1, subsample=0.8,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [46]:
r2_score(y_test,xgb_clf.predict(x_test_s) )

0.09979517952074168

In [47]:
xgb_clf = xgb.XGBRegressor(n_estimators=500,learning_rate=0.2,max_depth=3,gamma=0.2,subsample=0.8,reg_alpha=0.05,colsample_bytree=0.9)

xgb_clf.fit(x_train_n, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9, gamma=0.2, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.2, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=500, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0.05, reg_lambda=1, scale_pos_weight=1, subsample=0.8,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [48]:
r2_score(y_test,xgb_clf.predict(x_test_n) )

-2.425322770508802

# Attempt 2

In [49]:
df=pd.read_csv("train.csv",parse_dates=[12,15])

df["Delay"]=df["placement date"]-df["delivery date"]
df["Delay"]=df["Delay"]/ np.timedelta64(1,"D")

df["delivery type"]=df["delivery type"].fillna("Same Day")

del df["RID"]

del df["delivery date"]
del df["placement date"]
del df["departure city"]
del df["departure state"]
del df["address code"]

d={"Home Office":3,"Corporate":2,"Consumer":1}
df["segment"]=df["segment"].map(d)
d={"Central":1,"East":2,"West":3,"South":4}
df["location"]=df["location"].map(d)


d={"kariox":2,"qexty":1,"fynota":3}
df["class"]=df["class"].map(d)


h={}
n=1
for i in df.groupby("sub-class")["profit"].sum().sort_values().index:
    h[i]=n
    n+=1
df["sub-class"]=df["sub-class"].map(h)    


train_df=pd.get_dummies(df,columns=["delivery type"],drop_first=True)

y=df["profit"]
x=train_df.drop("profit",axis=1)


del x["id"]

In [98]:
x.Delay.unique()

array([  0., -31., -28., -30., -29.])

In [50]:
y

0         5.5512
1        13.3650
2         4.8609
3        12.7368
4        15.9792
          ...   
6695    152.0232
6696     89.9548
6697      0.9588
6698      2.3094
6699     10.0485
Name: profit, Length: 6700, dtype: float64

In [51]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import normalize

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

scalar=preprocessing.StandardScaler()
x_train_s=scalar.fit_transform(x_train)
x_test_s=scalar.fit_transform(x_test)

min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(x_train)
x_train_n = pd.DataFrame(np_scaled)

min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(x_test)
x_test_n = pd.DataFrame(np_scaled)

In [54]:
x_train_n

,0,1,2,3,4,5,6,7,8,9,10
0,0.250,0.000988,0.307692,0.666667,0.5,0.5,0.8125,1.0,0.0,0.0,1.0
1,0.000,0.027261,0.153846,1.000000,1.0,0.5,0.9375,1.0,0.0,0.0,1.0
2,0.375,0.004724,0.076923,0.000000,0.0,0.0,0.6875,1.0,0.0,0.0,1.0
3,0.000,0.004301,0.461538,0.333333,0.5,0.0,0.6250,1.0,0.0,0.0,1.0
4,0.000,0.001308,0.230769,0.333333,0.5,1.0,0.7500,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
5355,0.000,0.007183,0.153846,1.000000,0.5,0.0,0.5625,1.0,0.0,0.0,1.0
5356,0.875,0.000167,0.076923,0.333333,0.5,0.5,0.8125,1.0,0.0,1.0,0.0
5357,0.250,0.039372,0.461538,0.666667,0.0,0.0,0.0000,1.0,1.0,0.0,0.0
5358,0.000,0.034096,0.230769,0.666667,0.5,0.0,0.6250,1.0,0.0,0.0,1.0


In [55]:
import xgboost as xgb

xgb_clf = xgb.XGBRegressor(n_estimators=1000,learning_rate=0.2,max_depth=3,gamma=0.2,subsample=0.8,reg_alpha=0.05,colsample_bytree=0.9)

xgb_clf.fit(x_train, y_train)
r2_score(y_test,xgb_clf.predict(x_test) ),r2_score(y_train,xgb_clf.predict(x_train) )

(0.6395492377123362, 0.9964820685715499)

In [56]:
import xgboost as xgb
xgb_clf = xgb.XGBRegressor(n_estimators=1000,learning_rate=0.2,max_depth=3,gamma=0.2,subsample=0.8,reg_alpha=0.05,colsample_bytree=0.9)

xgb_clf.fit(x_train_s, y_train)
r2_score(y_test,xgb_clf.predict(x_test_s) ),r2_score(y_train,xgb_clf.predict(x_train_s) )

(0.757355591188657, 0.9965033557505416)

In [72]:
x

,discount,price,no of items,location,class,segment,sub-class,Delay,delivery type_Same Day,delivery type_Second Class,delivery type_Standard Class
0,0.2,16.448,2,1,2,1,13,0.0,0,0,1
1,0.0,29.700,5,1,2,1,14,0.0,0,0,1
2,0.0,14.730,3,1,1,1,9,0.0,0,0,1
3,0.0,43.920,3,1,2,1,6,0.0,0,0,1
4,0.0,66.580,2,1,2,1,11,0.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
6695,0.0,542.940,3,2,2,1,10,0.0,0,0,1
6696,0.2,276.784,2,3,2,2,14,0.0,0,0,1
6697,0.0,2.040,1,2,2,1,7,0.0,0,0,1
6698,0.1,207.846,3,2,1,2,12,0.0,0,1,0


# Attempt 3

In [93]:
df=pd.read_csv("train.csv",parse_dates=[12,15])

df["Delay"]=df["placement date"]-df["delivery date"]
df["Delay"]=df["Delay"]/ np.timedelta64(1,"D")

del df["RID"]
df["delivery type"]=df["delivery type"].fillna("Same Day")

del df["delivery date"]
del df["placement date"]
del df["departure city"]
del df["departure state"]

train_df=pd.get_dummies(df,columns=["location","class","segment","sub-class","delivery type"],drop_first=True)

y=df["profit"]
x=train_df.drop("profit",axis=1)


del x["id"]


In [83]:
x

,discount,price,no of items,address code,Delay,location_East,location_South,location_West,class_kariox,class_qexty,...,sub-class_phone,sub-class_shorts,sub-class_sweets,sub-class_tablet,sub-class_television,sub-class_tshirts,sub-class_watch,delivery type_Same Day,delivery type_Second Class,delivery type_Standard Class
0,0.2,16.448,2,77095,0.0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,1
1,0.0,29.700,5,48185,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,0.0,14.730,3,48185,0.0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1
3,0.0,43.920,3,48185,0.0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
4,0.0,66.580,2,48185,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6695,0.0,542.940,3,5408,0.0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
6696,0.2,276.784,2,95123,0.0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,1
6697,0.0,2.040,1,5408,0.0,1,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1
6698,0.1,207.846,3,10024,0.0,1,0,0,0,1,...,0,0,0,0,0,1,0,0,1,0


In [94]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import normalize

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

scalar=preprocessing.StandardScaler()
x_train_s=scalar.fit_transform(x_train)
x_test_s=scalar.fit_transform(x_test)

min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(x_train)
x_train_n = pd.DataFrame(np_scaled)

min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(x_test)
x_test_n = pd.DataFrame(np_scaled)



In [95]:
import xgboost as xgb

xgb_clf = xgb.XGBRegressor(n_estimators=1000,learning_rate=0.2,max_depth=3,gamma=0.2,subsample=0.8,reg_alpha=0.05,colsample_bytree=0.9)

xgb_clf.fit(x_train, y_train)
r2_score(y_test,xgb_clf.predict(x_test) ),r2_score(y_train,xgb_clf.predict(x_train) )



(0.6986595809653382, 0.9978190926684286)

In [96]:
import xgboost as xgb
xgb_clf = xgb.XGBRegressor(n_estimators=1000,learning_rate=0.2,max_depth=3,gamma=0.2,subsample=0.8,reg_alpha=0.05,colsample_bytree=0.9)

xgb_clf.fit(x_train_s, y_train)
r2_score(y_test,xgb_clf.predict(x_test_s) ),r2_score(y_train,xgb_clf.predict(x_train_s) )

(0.6971940122313931, 0.9978299307559364)

In [90]:
x_train

,discount,price,no of items,address code,Delay,location_East,location_South,location_West,class_kariox,class_qexty,...,sub-class_phone,sub-class_shorts,sub-class_sweets,sub-class_tablet,sub-class_television,sub-class_tshirts,sub-class_watch,delivery type_Same Day,delivery type_Second Class,delivery type_Standard Class
6026,0.0,64.960,2,10024,0.0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4072,0.0,79.920,4,90036,0.0,0,0,1,0,1,...,0,1,0,0,0,0,0,0,0,1
4177,0.0,79.920,4,98115,0.0,0,0,1,0,1,...,0,1,0,0,0,0,0,0,0,1
1598,0.0,167.940,3,11561,0.0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,0,1
2405,0.2,54.816,3,85204,0.0,0,0,1,1,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3405,0.7,7.380,5,27893,0.0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
5269,0.2,95.976,3,60610,-31.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3198,0.0,120.330,1,10009,0.0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
6627,0.0,316.000,4,98661,0.0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


# submission

In [85]:
df=pd.read_csv("train.csv",parse_dates=[12,15])

df["Delay"]=df["placement date"]-df["delivery date"]
df["Delay"]=df["Delay"]/ np.timedelta64(1,"D")

df["delivery type"]=df["delivery type"].fillna("Same Day")

# del df["RID"]

del df["delivery date"]
del df["placement date"]
del df["departure city"]
del df["departure state"]
# del df["address code"]

d={"Home Office":3,"Corporate":2,"Consumer":1}
df["segment"]=df["segment"].map(d)
d={"Central":1,"East":2,"West":3,"South":4}
df["location"]=df["location"].map(d)


d={"kariox":2,"qexty":1,"fynota":3}
df["class"]=df["class"].map(d)


h={}
n=1
for i in df.groupby("sub-class")["profit"].sum().sort_values().index:
    h[i]=n
    n+=1
df["sub-class"]=df["sub-class"].map(h)    


train_df=pd.get_dummies(df,columns=["delivery type"],drop_first=True)

y=df["profit"]
x=train_df.drop("profit",axis=1)


del x["id"]



test=pd.read_csv("test.csv",parse_dates=[11,14])

# del x_train["id"]

x_test=test.copy()

del x_test["id"]

x_test["Delay"]=x_test["placement date"]-x_test["delivery date"]
x_test["Delay"]=x_test["Delay"]/ np.timedelta64(1,"D")

del x_test["delivery date"]
del x_test["placement date"]
del x_test["departure city"]
del x_test["departure state"]

# del x_test["address code"]
d={"Home Office":3,"Corporate":2,"Consumer":1}
x_test["segment"]=x_test["segment"].map(d)
d={"Central":1,"East":2,"West":3,"South":4}
x_test["location"]=x_test["location"].map(d)


d={"kariox":2,"qexty":1,"fynota":3}
x_test["class"]=x_test["class"].map(d)



x_test["sub-class"]=x_test["sub-class"].map(h)    


x_test=pd.get_dummies(x_test,columns=["delivery type"],drop_first=True)

# del x_test["RID"]

In [59]:
x_test

,discount,price,no of items,location,class,segment,sub-class,RID,address code,Delay,delivery type_Same Day,delivery type_Second Class,delivery type_Standard Class
0,0.0,24.200,5,3,2,1,6,2408,94122,0.0,0,0,1
1,0.2,359.976,3,3,3,1,16,2409,94122,0.0,0,0,1
2,0.0,3.520,2,2,2,1,6,5425,6708,0.0,0,0,1
3,0.2,11.520,5,1,2,1,5,7408,60653,0.0,0,0,0
4,0.0,242.940,3,3,2,3,11,733,98115,0.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3289,0.0,31.920,4,2,2,1,11,1171,10035,0.0,0,1,0
3290,0.0,19.440,3,2,2,1,13,1170,10035,0.0,0,1,0
3291,0.0,272.940,3,2,2,1,11,1169,10035,0.0,0,1,0
3292,0.2,113.568,2,3,1,1,12,414,94110,0.0,0,0,1


In [87]:
x

,discount,price,no of items,location,class,segment,sub-class,RID,address code,Delay,delivery type_Same Day,delivery type_Second Class,delivery type_Standard Class
0,0.2,16.448,2,1,2,1,13,7981,77095,0.0,0,0,1
1,0.0,29.700,5,1,2,1,14,6334,48185,0.0,0,0,1
2,0.0,14.730,3,1,1,1,9,6333,48185,0.0,0,0,1
3,0.0,43.920,3,1,2,1,6,6332,48185,0.0,0,0,1
4,0.0,66.580,2,1,2,1,11,6331,48185,0.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6695,0.0,542.940,3,2,2,1,10,9389,5408,0.0,0,0,1
6696,0.2,276.784,2,3,2,2,14,9597,95123,0.0,0,0,1
6697,0.0,2.040,1,2,2,1,7,9390,5408,0.0,0,0,1
6698,0.1,207.846,3,2,1,2,12,440,10024,0.0,0,1,0


In [86]:
y

0         5.5512
1        13.3650
2         4.8609
3        12.7368
4        15.9792
          ...   
6695    152.0232
6696     89.9548
6697      0.9588
6698      2.3094
6699     10.0485
Name: profit, Length: 6700, dtype: float64

In [63]:
from sklearn import preprocessing
scalar=preprocessing.StandardScaler()
x_train_s=scalar.fit_transform(x)
x_test_s=scalar.fit_transform(x_test)

In [88]:
import xgboost as xgb
xgb_clf = xgb.XGBRegressor(n_estimators=1000,learning_rate=0.2,colsample_bytree= 0.9,gamma= 0.05,max_depth= 5,reg_alpha=0.05,subsample=0.8)
xgb_clf.fit(x, y)
r2_score(y,xgb_clf.predict(x) )

0.9998949211057685

In [74]:
import xgboost as xgb
xgb_clf = xgb.XGBRegressor(n_estimators=1000,learning_rate=0.2,max_depth=3,gamma=0.2,subsample=0.8,reg_alpha=0.05,colsample_bytree=0.9)

xgb_clf.fit(x, y)
r2_score(y,xgb_clf.predict(x) )

0.9974339036841527

In [89]:
final=pd.DataFrame(test["id"])

final["profit"]=xgb_clf.predict(x_test)

In [90]:
final

,id,profit
0,6701,7.790159
1,6702,40.794193
2,6703,2.637712
3,6704,1.852452
4,6705,13.851048
...,...,...
3289,9990,4.014146
3290,9991,9.676437
3291,9992,9.827170
3292,9993,4.682978


In [68]:
final.to_csv('submission4.csv', index=False)

In [84]:
r2_score(pd.read_csv("submission1.csv")["profit"],pd.read_csv("submission4.csv")["profit"])

0.9114507266066803

In [91]:
r2_score(final["profit"],pd.read_csv("submission1.csv")["profit"])

0.9415524467272655

# Attempt 4

In [92]:
df=pd.read_csv("train.csv",parse_dates=[12,15])

df["Delay"]=df["placement date"]-df["delivery date"]
df["Delay"]=df["Delay"]/ np.timedelta64(1,"D")

df["delivery type"]=df["delivery type"].fillna("Same Day")

# del df["RID"]

del df["delivery date"]
del df["placement date"]
del df["departure city"]
del df["departure state"]
# del df["address code"]

d={"Home Office":3,"Corporate":2,"Consumer":1}
df["segment"]=df["segment"].map(d)
d={"Central":1,"East":2,"West":3,"South":4}
df["location"]=df["location"].map(d)


d={"kariox":2,"qexty":1,"fynota":3}
df["class"]=df["class"].map(d)


h={}
n=1
for i in df.groupby("sub-class")["profit"].sum().sort_values().index:
    h[i]=n
    n+=1
df["sub-class"]=df["sub-class"].map(h)    


train_df=pd.get_dummies(df,columns=["delivery type"],drop_first=True)

y=df["profit"]
x=train_df.drop("profit",axis=1)


del x["id"]

In [93]:
x

,discount,price,no of items,location,class,segment,sub-class,RID,address code,Delay,delivery type_Same Day,delivery type_Second Class,delivery type_Standard Class
0,0.2,16.448,2,1,2,1,13,7981,77095,0.0,0,0,1
1,0.0,29.700,5,1,2,1,14,6334,48185,0.0,0,0,1
2,0.0,14.730,3,1,1,1,9,6333,48185,0.0,0,0,1
3,0.0,43.920,3,1,2,1,6,6332,48185,0.0,0,0,1
4,0.0,66.580,2,1,2,1,11,6331,48185,0.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6695,0.0,542.940,3,2,2,1,10,9389,5408,0.0,0,0,1
6696,0.2,276.784,2,3,2,2,14,9597,95123,0.0,0,0,1
6697,0.0,2.040,1,2,2,1,7,9390,5408,0.0,0,0,1
6698,0.1,207.846,3,2,1,2,12,440,10024,0.0,0,1,0


In [94]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import normalize

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [95]:
import xgboost as xgb
xgb_clf = xgb.XGBRegressor(n_estimators=1000,learning_rate=0.2,colsample_bytree= 0.9,gamma= 0.05,max_depth= 5,reg_alpha=0.05,subsample=0.8)
xgb_clf.fit(x_train, y_train)
r2_score(y_train,xgb_clf.predict(x_train) )

0.9999509576361606

In [96]:
r2_score(y_test,xgb_clf.predict(x_test) )

0.5741044011733725

# Attempt 5

In [109]:
df=pd.read_csv("train.csv",parse_dates=[12,15])

df["Delay"]=df["placement date"]-df["delivery date"]
df["Delay"]=df["Delay"]/ np.timedelta64(1,"D")

df["delivery type"]=df["delivery type"].fillna("Same Day")

del df["RID"]
# del df["address code"]

del df["delivery date"]
del df["placement date"]
del df["departure city"]
del df["departure state"]

train_df=pd.get_dummies(df,columns=["location","class","segment","sub-class","delivery type"],drop_first=True)

y=df["profit"]
x=train_df.drop("profit",axis=1)


del x["id"]

In [110]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import normalize


x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [111]:
import xgboost as xgb
xgb_clf = xgb.XGBRegressor(n_estimators=1000,learning_rate=0.2,colsample_bytree= 0.9,gamma= 0.05,max_depth= 5,reg_alpha=0.05,subsample=0.8)
xgb_clf.fit(x_train, y_train)
r2_score(y_train,xgb_clf.predict(x_train) )

0.9999289792118277

In [112]:
r2_score(y_test,xgb_clf.predict(x_test) )

0.8169499162431388

# Submission

In [42]:
df=pd.read_csv("train.csv",parse_dates=[12,15])

df["Delay"]=df["placement date"]-df["delivery date"]
df["Delay"]=df["Delay"]/ np.timedelta64(1,"D")

df["delivery type"]=df["delivery type"].fillna("Same Day")

# del df["RID"]
# del df["address code"]

del df["delivery date"]
del df["placement date"]
del df["departure city"]
del df["departure state"]

train_df=pd.get_dummies(df,columns=["location","class","segment","sub-class","delivery type"],drop_first=True)

y=df["profit"]
x=train_df.drop("profit",axis=1)


del x["id"]





test=pd.read_csv("test.csv",parse_dates=[11,14])


x_test=test.copy()

del x_test["id"]

x_test["Delay"]=x_test["placement date"]-x_test["delivery date"]
x_test["Delay"]=x_test["Delay"]/ np.timedelta64(1,"D")

del x_test["delivery date"]
del x_test["placement date"]
del x_test["departure city"]
del x_test["departure state"]

x_test=pd.get_dummies(x_test,columns=["location","class","segment","sub-class","delivery type"],drop_first=True)
# del x_test["address code"]
# del x_test["RID"]

In [43]:
x

,discount,price,no of items,RID,address code,Delay,location_East,location_South,location_West,class_kariox,...,sub-class_phone,sub-class_shorts,sub-class_sweets,sub-class_tablet,sub-class_television,sub-class_tshirts,sub-class_watch,delivery type_Same Day,delivery type_Second Class,delivery type_Standard Class
0,0.2,16.448,2,7981,77095,0.0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,1
1,0.0,29.700,5,6334,48185,0.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,0.0,14.730,3,6333,48185,0.0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
3,0.0,43.920,3,6332,48185,0.0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1
4,0.0,66.580,2,6331,48185,0.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6695,0.0,542.940,3,9389,5408,0.0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1
6696,0.2,276.784,2,9597,95123,0.0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,1
6697,0.0,2.040,1,9390,5408,0.0,1,0,0,1,...,0,0,0,0,0,0,1,0,0,1
6698,0.1,207.846,3,440,10024,0.0,1,0,0,0,...,0,0,0,0,0,1,0,0,1,0


In [44]:
x_test

,discount,price,no of items,RID,address code,Delay,location_East,location_South,location_West,class_kariox,...,sub-class_phone,sub-class_shorts,sub-class_sweets,sub-class_tablet,sub-class_television,sub-class_tshirts,sub-class_watch,delivery type_Same Day,delivery type_Second Class,delivery type_Standard Class
0,0.0,24.200,5,2408,94122,0.0,0,0,1,1,...,0,0,0,0,1,0,0,0,0,1
1,0.2,359.976,3,2409,94122,0.0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,0.0,3.520,2,5425,6708,0.0,1,0,0,1,...,0,0,0,0,1,0,0,0,0,1
3,0.2,11.520,5,7408,60653,0.0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
4,0.0,242.940,3,733,98115,0.0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3289,0.0,31.920,4,1171,10035,0.0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
3290,0.0,19.440,3,1170,10035,0.0,1,0,0,1,...,1,0,0,0,0,0,0,0,1,0
3291,0.0,272.940,3,1169,10035,0.0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
3292,0.2,113.568,2,414,94110,0.0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1


In [45]:
import xgboost as xgb
#xgb_clf = xgb.XGBRegressor(n_estimators=1000,learning_rate=0.2,colsample_bytree= 0.9,gamma= 0.05,max_depth= 5,reg_alpha=0.05,subsample=0.8)
xgb_clf = xgb.XGBRegressor(n_estimators=500,learning_rate=0.2,max_depth=3,gamma=0.2,subsample=0.8,reg_alpha=0.5,colsample_bytree=0.9)
xgb_clf.fit(x, y)
r2_score(y,xgb_clf.predict(x) )

0.9942360448251008

In [46]:
final=pd.DataFrame(test["id"])

final["profit"]=xgb_clf.predict(x_test)

In [47]:
r2_score(pd.read_csv("submission1.csv")["profit"],final["profit"])

0.9733292901749842

In [41]:
final.to_csv('submission8.csv', index=False)

In [130]:
r2_score(pd.read_csv("submission1.csv")["profit"],pd.read_csv("submission6.csv")["profit"])

1.0

In [39]:
final

,id,profit
0,6701,4.980422
1,6702,31.695112
2,6703,2.405542
3,6704,2.185692
4,6705,20.897997
...,...,...
3289,9990,2.027780
3290,9991,9.546029
3291,9992,22.908361
3292,9993,0.593660


In [40]:
pd.read_csv("submission1.csv")

,id,profit
0,6701,7.612872
1,6702,39.257774
2,6703,-0.815404
3,6704,5.485227
4,6705,23.871029
...,...,...
3289,9990,-1.171187
3290,9991,9.648065
3291,9992,28.005226
3292,9993,7.939437


# Attempt 5

In [4]:
df=pd.read_csv("train.csv",parse_dates=[12,15])

df["Delay"]=df["placement date"]-df["delivery date"]
df["Delay"]=df["Delay"]/ np.timedelta64(1,"D")

df["delivery type"]=df["delivery type"].fillna("Same Day")

del df["RID"]
del df["address code"]
df["price/item"]=df["price"]/df["no of items"]
del df["delivery date"]
del df["placement date"]
del df["departure city"]
del df["departure state"]

train_df=pd.get_dummies(df,columns=["location","class","segment","sub-class","delivery type"],drop_first=True)

y=df["profit"]
x=train_df.drop("profit",axis=1)


del x["id"]





test=pd.read_csv("test.csv",parse_dates=[11,14])


x_test=test.copy()

del x_test["id"]

x_test["Delay"]=x_test["placement date"]-x_test["delivery date"]
x_test["Delay"]=x_test["Delay"]/ np.timedelta64(1,"D")

del x_test["delivery date"]
del x_test["placement date"]
del x_test["departure city"]
del x_test["departure state"]
x_test["price/item"]=x_test["price"]/x_test["no of items"]
x_test=pd.get_dummies(x_test,columns=["location","class","segment","sub-class","delivery type"],drop_first=True)
del x_test["address code"]
del x_test["RID"]

In [5]:
x

,discount,price,no of items,Delay,price/item,location_East,location_South,location_West,class_kariox,class_qexty,...,sub-class_phone,sub-class_shorts,sub-class_sweets,sub-class_tablet,sub-class_television,sub-class_tshirts,sub-class_watch,delivery type_Same Day,delivery type_Second Class,delivery type_Standard Class
0,0.2,16.448,2,0.0,8.224,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,1
1,0.0,29.700,5,0.0,5.940,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,0.0,14.730,3,0.0,4.910,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1
3,0.0,43.920,3,0.0,14.640,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
4,0.0,66.580,2,0.0,33.290,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6695,0.0,542.940,3,0.0,180.980,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
6696,0.2,276.784,2,0.0,138.392,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,1
6697,0.0,2.040,1,0.0,2.040,1,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1
6698,0.1,207.846,3,0.0,69.282,1,0,0,0,1,...,0,0,0,0,0,1,0,0,1,0


In [6]:
x_test

,discount,price,no of items,Delay,price/item,location_East,location_South,location_West,class_kariox,class_qexty,...,sub-class_phone,sub-class_shorts,sub-class_sweets,sub-class_tablet,sub-class_television,sub-class_tshirts,sub-class_watch,delivery type_Same Day,delivery type_Second Class,delivery type_Standard Class
0,0.0,24.200,5,0.0,4.840,0,0,1,1,0,...,0,0,0,0,1,0,0,0,0,1
1,0.2,359.976,3,0.0,119.992,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0.0,3.520,2,0.0,1.760,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
3,0.2,11.520,5,0.0,2.304,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
4,0.0,242.940,3,0.0,80.980,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3289,0.0,31.920,4,0.0,7.980,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3290,0.0,19.440,3,0.0,6.480,1,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
3291,0.0,272.940,3,0.0,90.980,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3292,0.2,113.568,2,0.0,56.784,0,0,1,0,1,...,0,0,0,0,0,1,0,0,0,1


In [93]:
xgb_clf = xgb.XGBRegressor(n_estimators=1000,learning_rate=0.1,max_depth=3,gamma=0.2,subsample=0.8,reg_alpha=0.5,colsample_bytree=0.9)
xgb_clf.fit(x, y)
r2_score(y,xgb_clf.predict(x) )

0.9958268967799387

In [17]:
import xgboost as xgb
xgb_clf = xgb.XGBRegressor(n_estimators=1000,max_depth=3,gamma=0.2,subsample=0.8,reg_alpha=0.5,colsample_bytree=0.9)
xgb_clf.fit(
    x,
    y,
    eval_metric="rmse",
    eval_set=[(x, y), (x, y)],
    verbose=True,
    early_stopping_rounds = 10)

[0]	validation_0-rmse:205.38518	validation_1-rmse:205.38518
[1]	validation_0-rmse:188.01056	validation_1-rmse:188.01056
[2]	validation_0-rmse:162.81883	validation_1-rmse:162.81883
[3]	validation_0-rmse:148.87566	validation_1-rmse:148.87566
[4]	validation_0-rmse:140.57797	validation_1-rmse:140.57797
[5]	validation_0-rmse:130.39414	validation_1-rmse:130.39414
[6]	validation_0-rmse:119.55745	validation_1-rmse:119.55745
[7]	validation_0-rmse:117.17532	validation_1-rmse:117.17532
[8]	validation_0-rmse:112.00243	validation_1-rmse:112.00243
[9]	validation_0-rmse:102.80603	validation_1-rmse:102.80603
[10]	validation_0-rmse:99.99976	validation_1-rmse:99.99976
[11]	validation_0-rmse:97.82172	validation_1-rmse:97.82172
[12]	validation_0-rmse:97.27062	validation_1-rmse:97.27062
[13]	validation_0-rmse:96.33130	validation_1-rmse:96.33130
[14]	validation_0-rmse:92.75822	validation_1-rmse:92.75822
[15]	validation_0-rmse:86.10085	validation_1-rmse:86.10085
[16]	validation_0-rmse:84.42673	validation_1-r

[139]	validation_0-rmse:26.75546	validation_1-rmse:26.75546
[140]	validation_0-rmse:26.74059	validation_1-rmse:26.74059
[141]	validation_0-rmse:26.62372	validation_1-rmse:26.62372
[142]	validation_0-rmse:26.42157	validation_1-rmse:26.42157
[143]	validation_0-rmse:26.30285	validation_1-rmse:26.30285
[144]	validation_0-rmse:26.17735	validation_1-rmse:26.17735
[145]	validation_0-rmse:26.07319	validation_1-rmse:26.07319
[146]	validation_0-rmse:25.95489	validation_1-rmse:25.95489
[147]	validation_0-rmse:25.76841	validation_1-rmse:25.76841
[148]	validation_0-rmse:25.69426	validation_1-rmse:25.69426
[149]	validation_0-rmse:25.52939	validation_1-rmse:25.52939
[150]	validation_0-rmse:25.32737	validation_1-rmse:25.32737
[151]	validation_0-rmse:25.24630	validation_1-rmse:25.24630
[152]	validation_0-rmse:25.18071	validation_1-rmse:25.18071
[153]	validation_0-rmse:25.06818	validation_1-rmse:25.06818
[154]	validation_0-rmse:25.03084	validation_1-rmse:25.03084
[155]	validation_0-rmse:25.00089	validat

[276]	validation_0-rmse:17.97560	validation_1-rmse:17.97560
[277]	validation_0-rmse:17.94337	validation_1-rmse:17.94337
[278]	validation_0-rmse:17.90333	validation_1-rmse:17.90333
[279]	validation_0-rmse:17.87598	validation_1-rmse:17.87598
[280]	validation_0-rmse:17.86683	validation_1-rmse:17.86683
[281]	validation_0-rmse:17.81053	validation_1-rmse:17.81053
[282]	validation_0-rmse:17.78661	validation_1-rmse:17.78661
[283]	validation_0-rmse:17.73762	validation_1-rmse:17.73762
[284]	validation_0-rmse:17.69905	validation_1-rmse:17.69905
[285]	validation_0-rmse:17.65825	validation_1-rmse:17.65825
[286]	validation_0-rmse:17.62127	validation_1-rmse:17.62127
[287]	validation_0-rmse:17.57835	validation_1-rmse:17.57835
[288]	validation_0-rmse:17.55159	validation_1-rmse:17.55159
[289]	validation_0-rmse:17.51862	validation_1-rmse:17.51862
[290]	validation_0-rmse:17.49425	validation_1-rmse:17.49425
[291]	validation_0-rmse:17.41598	validation_1-rmse:17.41598
[292]	validation_0-rmse:17.37363	validat

[413]	validation_0-rmse:14.06396	validation_1-rmse:14.06396
[414]	validation_0-rmse:14.04629	validation_1-rmse:14.04629
[415]	validation_0-rmse:14.03140	validation_1-rmse:14.03140
[416]	validation_0-rmse:14.01651	validation_1-rmse:14.01651
[417]	validation_0-rmse:13.98788	validation_1-rmse:13.98788
[418]	validation_0-rmse:13.97853	validation_1-rmse:13.97853
[419]	validation_0-rmse:13.95724	validation_1-rmse:13.95724
[420]	validation_0-rmse:13.92092	validation_1-rmse:13.92092
[421]	validation_0-rmse:13.89662	validation_1-rmse:13.89662
[422]	validation_0-rmse:13.85955	validation_1-rmse:13.85955
[423]	validation_0-rmse:13.84140	validation_1-rmse:13.84140
[424]	validation_0-rmse:13.81080	validation_1-rmse:13.81080
[425]	validation_0-rmse:13.77926	validation_1-rmse:13.77926
[426]	validation_0-rmse:13.75859	validation_1-rmse:13.75859
[427]	validation_0-rmse:13.75249	validation_1-rmse:13.75249
[428]	validation_0-rmse:13.73966	validation_1-rmse:13.73966
[429]	validation_0-rmse:13.72736	validat

[550]	validation_0-rmse:11.53365	validation_1-rmse:11.53365
[551]	validation_0-rmse:11.52386	validation_1-rmse:11.52386
[552]	validation_0-rmse:11.50029	validation_1-rmse:11.50029
[553]	validation_0-rmse:11.49220	validation_1-rmse:11.49220
[554]	validation_0-rmse:11.48034	validation_1-rmse:11.48034
[555]	validation_0-rmse:11.47490	validation_1-rmse:11.47490
[556]	validation_0-rmse:11.45888	validation_1-rmse:11.45888
[557]	validation_0-rmse:11.44837	validation_1-rmse:11.44837
[558]	validation_0-rmse:11.43366	validation_1-rmse:11.43366
[559]	validation_0-rmse:11.40667	validation_1-rmse:11.40667
[560]	validation_0-rmse:11.39527	validation_1-rmse:11.39527
[561]	validation_0-rmse:11.37547	validation_1-rmse:11.37547
[562]	validation_0-rmse:11.36909	validation_1-rmse:11.36909
[563]	validation_0-rmse:11.35237	validation_1-rmse:11.35237
[564]	validation_0-rmse:11.34827	validation_1-rmse:11.34827
[565]	validation_0-rmse:11.33262	validation_1-rmse:11.33262
[566]	validation_0-rmse:11.32932	validat

[687]	validation_0-rmse:9.87099	validation_1-rmse:9.87099
[688]	validation_0-rmse:9.85133	validation_1-rmse:9.85133
[689]	validation_0-rmse:9.84494	validation_1-rmse:9.84494
[690]	validation_0-rmse:9.83019	validation_1-rmse:9.83019
[691]	validation_0-rmse:9.81479	validation_1-rmse:9.81479
[692]	validation_0-rmse:9.80825	validation_1-rmse:9.80825
[693]	validation_0-rmse:9.79250	validation_1-rmse:9.79250
[694]	validation_0-rmse:9.78111	validation_1-rmse:9.78111
[695]	validation_0-rmse:9.76174	validation_1-rmse:9.76174
[696]	validation_0-rmse:9.74791	validation_1-rmse:9.74791
[697]	validation_0-rmse:9.74071	validation_1-rmse:9.74071
[698]	validation_0-rmse:9.72792	validation_1-rmse:9.72792
[699]	validation_0-rmse:9.72681	validation_1-rmse:9.72681
[700]	validation_0-rmse:9.71370	validation_1-rmse:9.71370
[701]	validation_0-rmse:9.70019	validation_1-rmse:9.70019
[702]	validation_0-rmse:9.68450	validation_1-rmse:9.68450
[703]	validation_0-rmse:9.67952	validation_1-rmse:9.67952
[704]	validati

[829]	validation_0-rmse:8.61068	validation_1-rmse:8.61068
[830]	validation_0-rmse:8.60666	validation_1-rmse:8.60666
[831]	validation_0-rmse:8.60016	validation_1-rmse:8.60016
[832]	validation_0-rmse:8.59436	validation_1-rmse:8.59436
[833]	validation_0-rmse:8.59173	validation_1-rmse:8.59173
[834]	validation_0-rmse:8.58483	validation_1-rmse:8.58483
[835]	validation_0-rmse:8.58119	validation_1-rmse:8.58119
[836]	validation_0-rmse:8.57952	validation_1-rmse:8.57952
[837]	validation_0-rmse:8.56921	validation_1-rmse:8.56921
[838]	validation_0-rmse:8.56094	validation_1-rmse:8.56094
[839]	validation_0-rmse:8.54230	validation_1-rmse:8.54230
[840]	validation_0-rmse:8.53800	validation_1-rmse:8.53800
[841]	validation_0-rmse:8.53149	validation_1-rmse:8.53149
[842]	validation_0-rmse:8.52918	validation_1-rmse:8.52918
[843]	validation_0-rmse:8.52028	validation_1-rmse:8.52028
[844]	validation_0-rmse:8.51557	validation_1-rmse:8.51557
[845]	validation_0-rmse:8.50659	validation_1-rmse:8.50659
[846]	validati

[971]	validation_0-rmse:7.68245	validation_1-rmse:7.68245
[972]	validation_0-rmse:7.67934	validation_1-rmse:7.67934
[973]	validation_0-rmse:7.67140	validation_1-rmse:7.67140
[974]	validation_0-rmse:7.66175	validation_1-rmse:7.66175
[975]	validation_0-rmse:7.65279	validation_1-rmse:7.65279
[976]	validation_0-rmse:7.63988	validation_1-rmse:7.63988
[977]	validation_0-rmse:7.62981	validation_1-rmse:7.62981
[978]	validation_0-rmse:7.62352	validation_1-rmse:7.62352
[979]	validation_0-rmse:7.62066	validation_1-rmse:7.62066
[980]	validation_0-rmse:7.61601	validation_1-rmse:7.61601
[981]	validation_0-rmse:7.61173	validation_1-rmse:7.61173
[982]	validation_0-rmse:7.60670	validation_1-rmse:7.60670
[983]	validation_0-rmse:7.60094	validation_1-rmse:7.60094
[984]	validation_0-rmse:7.59817	validation_1-rmse:7.59817
[985]	validation_0-rmse:7.59249	validation_1-rmse:7.59249
[986]	validation_0-rmse:7.58799	validation_1-rmse:7.58799
[987]	validation_0-rmse:7.58292	validation_1-rmse:7.58292
[988]	validati

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9, gamma=0.2, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0.5, reg_lambda=1, scale_pos_weight=1, subsample=0.8,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [18]:
final=pd.DataFrame(test["id"])

final["profit"]=xgb_clf.predict(x_test)

In [19]:
r2_score(pd.read_csv("submission1.csv")["profit"],final["profit"])

0.9630389405943611

In [96]:
final.to_csv('submission9.csv', index=False)

# Attempt 10

# ANN

In [133]:
df=pd.read_csv("train.csv",parse_dates=[12,15])

df["Delay"]=df["placement date"]-df["delivery date"]
df["Delay"]=df["Delay"]/ np.timedelta64(1,"D")

df["delivery type"]=df["delivery type"].fillna("Same Day")

del df["RID"]
del df["address code"]
df["price/item"]=df["price"]/df["no of items"]
del df["delivery date"]
del df["placement date"]
del df["departure city"]
del df["departure state"]

train_df=pd.get_dummies(df,columns=["location","class","segment","sub-class","delivery type"],drop_first=True)

y=df["profit"]
x=train_df.drop("profit",axis=1)


del x["id"]





test=pd.read_csv("test.csv",parse_dates=[11,14])


x_test=test.copy()

del x_test["id"]

x_test["Delay"]=x_test["placement date"]-x_test["delivery date"]
x_test["Delay"]=x_test["Delay"]/ np.timedelta64(1,"D")

del x_test["delivery date"]
del x_test["placement date"]
del x_test["departure city"]
del x_test["departure state"]
x_test["price/item"]=x_test["price"]/x_test["no of items"]
x_test=pd.get_dummies(x_test,columns=["location","class","segment","sub-class","delivery type"],drop_first=True)
del x_test["address code"]
del x_test["RID"]

In [106]:
x_train

,discount,price,no of items,Delay,price/item,location_East,location_South,location_West,class_kariox,class_qexty,...,sub-class_phone,sub-class_shorts,sub-class_sweets,sub-class_tablet,sub-class_television,sub-class_tshirts,sub-class_watch,delivery type_Same Day,delivery type_Second Class,delivery type_Standard Class
161,0.2,30.768,2,0.0,15.384,0,1,0,0,1,...,0,1,0,0,0,0,0,0,0,1
3868,0.0,195.640,4,0.0,48.910,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
2031,0.0,9.840,3,0.0,3.280,1,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
5976,0.2,113.568,2,0.0,56.784,1,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1113,0.2,13.160,5,0.0,2.632,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1619,0.2,7.856,2,0.0,3.928,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
6039,0.0,1117.920,4,0.0,279.480,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
5391,0.2,185.376,2,0.0,92.688,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1129,0.0,4.020,2,0.0,2.010,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,1


In [134]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import normalize


x_train,test_x,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [135]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.losses import mean_squared_error
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from tensorflow import keras



In [136]:
from tensorflow.keras.models import Sequential


from tensorflow.keras.layers import InputLayer, Dense ,Dropout

In [137]:
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow.keras.callbacks

In [138]:
model = Sequential()

model.add(Dense(128, input_dim=31, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(32, activation='relu'))
# model.add(Dense(31, activation='relu'))
model.add(Dense(31, activation='linear'))

In [139]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_52 (Dense)             (None, 128)               4096      
_________________________________________________________________
dense_53 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_54 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_55 (Dense)             (None, 31)                1023      
Total params: 15,455
Trainable params: 15,455
Non-trainable params: 0
_________________________________________________________________


In [140]:
model.compile(optimizer="Adam",loss="mse",metrics=["mse"])

In [141]:
model_history = model.fit(x_train, y_train, validation_data=(test_x, y_test), epochs=100,callbacks=[keras.callbacks.EarlyStopping(monitor="val_mse",patience=10)])

Epoch 1/100
168/168 [==============================] - 3s 14ms/step - loss: 79875.3373 - mse: 79875.3373 - val_loss: 32816.6914 - val_mse: 32816.6914
Epoch 2/100
168/168 [==============================] - 2s 13ms/step - loss: 55936.1312 - mse: 55936.1312 - val_loss: 28391.1074 - val_mse: 28391.1074
Epoch 3/100
168/168 [==============================] - 3s 18ms/step - loss: 52222.6239 - mse: 52222.6239 - val_loss: 23890.0547 - val_mse: 23890.0547
Epoch 4/100
168/168 [==============================] - 2s 12ms/step - loss: 49895.8689 - mse: 49895.8689 - val_loss: 25088.8203 - val_mse: 25088.8203
Epoch 5/100
168/168 [==============================] - 3s 16ms/step - loss: 44430.9185 - mse: 44430.9185 - val_loss: 28448.5488 - val_mse: 28448.5488
Epoch 6/100
168/168 [==============================] - 2s 13ms/step - loss: 39939.4955 - mse: 39939.4955 - val_loss: 22283.5215 - val_mse: 22283.5215
Epoch 7/100
168/168 [==============================] - 1s 9ms/step - loss: 67789.9017 - mse: 67789.9

In [150]:
new=pd.DataFrame(model.predict(x_train))

In [156]:
new.columns=['discount', 'price', 'no of items', 'Delay', 'price/item',
       'location_East', 'location_South', 'location_West', 'class_kariox',
       'class_qexty', 'segment_Corporate', 'segment_Home Office',
       'sub-class_charger', 'sub-class_chocolates', 'sub-class_colddrinks',
       'sub-class_fastfood', 'sub-class_headset', 'sub-class_hoodies',
       'sub-class_laptop', 'sub-class_lighting', 'sub-class_pants',
       'sub-class_phone', 'sub-class_shorts', 'sub-class_sweets',
       'sub-class_tablet', 'sub-class_television', 'sub-class_tshirts',
       'sub-class_watch', 'delivery type_Same Day',
       'delivery type_Second Class', 'delivery type_Standard Class']

In [157]:
new

,discount,price,no of items,Delay,price/item,location_East,location_South,location_West,class_kariox,class_qexty,...,sub-class_phone,sub-class_shorts,sub-class_sweets,sub-class_tablet,sub-class_television,sub-class_tshirts,sub-class_watch,delivery type_Same Day,delivery type_Second Class,delivery type_Standard Class
0,5.163005,6.904469,5.299429,5.280216,6.416720,5.547986,5.789615,4.817672,5.571560,4.800690,...,5.658717,5.173188,6.627787,6.416132,5.636840,4.803174,4.220135,5.858639,5.122919,6.832135
1,2.699041,4.397734,3.758140,3.453829,3.568099,3.001805,3.146720,2.898358,3.628082,2.639917,...,4.261396,3.129393,3.644056,4.833654,3.335262,2.321802,2.095836,3.460113,3.661078,3.766956
2,31.332155,31.284224,31.596544,38.281345,24.672304,33.656368,35.489723,32.225922,33.578526,38.071999,...,29.645435,36.743118,35.423149,34.764084,33.320614,31.499182,32.378689,30.937473,31.768536,24.189907
3,6.745952,8.749110,6.776236,6.761006,9.109772,6.510439,6.017474,5.558880,6.590409,4.398042,...,7.316852,5.621666,7.594383,8.339849,6.013392,5.380276,4.224947,6.591646,6.754486,10.251044
4,5.709731,7.385448,7.434588,7.762030,6.865150,4.319072,7.100079,5.324285,7.031250,3.543273,...,7.712413,5.260252,3.891197,7.555735,7.150336,6.704717,4.211470,4.579053,8.731851,6.364162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5355,1.856197,3.149218,3.597233,3.592342,0.731108,0.715556,3.334630,1.633422,3.250550,1.359279,...,3.410329,3.759134,1.447460,4.447559,3.168027,1.336383,-0.025075,1.379285,3.060911,-0.012116
5356,13.896696,16.775646,19.004194,16.522095,22.267839,18.243528,11.832087,12.927695,12.447957,11.468032,...,20.494549,8.353544,11.066809,16.789455,15.966460,14.898495,13.723588,12.377363,19.158794,20.803482
5357,9.787208,11.355170,13.167632,11.973401,15.166687,13.187760,8.188944,9.279942,9.122731,8.015120,...,13.925838,6.574706,7.784069,10.794954,10.624082,10.797524,9.540006,8.847638,12.721167,14.312160
5358,44.632793,44.395847,44.465313,53.878002,36.143867,47.986103,50.621197,45.997017,47.668430,53.495071,...,41.698875,51.003918,49.732491,47.689693,47.083614,45.555004,46.573227,43.953270,44.915943,35.605999


In [155]:
x_train.columns

Index(['discount', 'price', 'no of items', 'Delay', 'price/item',
       'location_East', 'location_South', 'location_West', 'class_kariox',
       'class_qexty', 'segment_Corporate', 'segment_Home Office',
       'sub-class_charger', 'sub-class_chocolates', 'sub-class_colddrinks',
       'sub-class_fastfood', 'sub-class_headset', 'sub-class_hoodies',
       'sub-class_laptop', 'sub-class_lighting', 'sub-class_pants',
       'sub-class_phone', 'sub-class_shorts', 'sub-class_sweets',
       'sub-class_tablet', 'sub-class_television', 'sub-class_tshirts',
       'sub-class_watch', 'delivery type_Same Day',
       'delivery type_Second Class', 'delivery type_Standard Class'],
      dtype='object')

In [158]:
import xgboost as xgb
xgb_clf = xgb.XGBRegressor(n_estimators=1000,max_depth=3,gamma=0.2,subsample=0.8,reg_alpha=0.5,colsample_bytree=0.9)
xgb_clf.fit(new,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9, gamma=0.2, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0.5, reg_lambda=1, scale_pos_weight=1, subsample=0.8,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [159]:
r2_score(y_train,xgb_clf.predict(x_train))

-2.9026803728843813

In [119]:
final=pd.DataFrame(test["id"])

final["profit"]=model.predict(x_test)



In [120]:
r2_score(pd.read_csv("submission1.csv")["profit"],final["profit"])

0.14309760336457855

In [121]:
final

,id,profit
0,6701,5.470200
1,6702,14.512079
2,6703,3.291481
3,6704,3.997918
4,6705,7.127499
...,...,...
3289,9990,3.479148
3290,9991,6.357423
3291,9992,8.908507
3292,9993,-4.264932


# Attempt 11

In [56]:
df=pd.read_csv("train.csv",parse_dates=[12,15])

df["Delay"]=df["placement date"]-df["delivery date"]
df["Delay"]=df["Delay"]/ np.timedelta64(1,"D")

df["delivery type"]=df["delivery type"].fillna("Same Day")

del df["RID"]
del df["address code"]
df["price/item"]=df["price"]/df["no of items"]
del df["delivery date"]

del df["departure city"]
del df["departure state"]

cls={'fynota': 0.4944459733177522,
 'kariox': 0.42633614865384356,
 'qexty': 0.07921792671863703}
sub={'fastfood': 0.16383500520216696,
 'colddrinks': 0.16253265115754612,
 'chocolates': 0.13524541261128278,
 'headset': 0.11589063251174975,
 'phone': 0.11288672550496903,
 'tshirts': 0.09711869858695824,
 'laptop': 0.07165886802896806,
 'battery': 0.05346435567446218,
 'shorts': 0.04592247899913388,
 'sweets': 0.03283290434675645,
 'watch': 0.0287734720236173,
 'television': 0.02204642639318131,
 'tablet': 0.0194845226563341,
 'lighting': 0.0033293699483883522,
 'charger': -0.0011982240878269281,
 'pants': -0.014806596757743028,
 'hoodies': -0.049016654109711934}

df["class"]=df["class"].map(cls)
df["sub-class"]=df["sub-class"].map(sub)
df["month"]=df["placement date"].dt.month
df["year"]=df["placement date"].dt.year
del df["placement date"]

train_df=pd.get_dummies(df,columns=["location","segment","delivery type"],drop_first=True)

y=df["profit"]
x=train_df.drop("profit",axis=1)


del x["id"]





test=pd.read_csv("test.csv",parse_dates=[11,14])


x_test=test.copy()

del x_test["id"]

x_test["Delay"]=x_test["placement date"]-x_test["delivery date"]
x_test["Delay"]=x_test["Delay"]/ np.timedelta64(1,"D")

x_test["class"]=x_test["class"].map(cls)
x_test["sub-class"]=x_test["sub-class"].map(sub)
x_test["price/item"]=x_test["price"]/x_test["no of items"]
x_test["month"]=x_test["placement date"].dt.month
x_test["year"]=x_test["placement date"].dt.year

del x_test["delivery date"]
del x_test["placement date"]
del x_test["departure city"]
del x_test["departure state"]

x_test=pd.get_dummies(x_test,columns=["location","segment","delivery type"],drop_first=True)
del x_test["address code"]
del x_test["RID"]

In [57]:
x_test

,discount,price,no of items,class,sub-class,Delay,price/item,month,year,location_East,location_South,location_West,segment_Corporate,segment_Home Office,delivery type_Same Day,delivery type_Second Class,delivery type_Standard Class
0,0.0,24.200,5,0.426336,0.022046,0.0,4.840,1,2021,0,0,1,0,0,0,0,1
1,0.2,359.976,3,0.494446,0.162533,0.0,119.992,1,2021,0,0,1,0,0,0,0,1
2,0.0,3.520,2,0.426336,0.022046,0.0,1.760,1,2021,1,0,0,0,0,0,0,1
3,0.2,11.520,5,0.426336,0.019485,0.0,2.304,1,2021,0,0,0,0,0,0,0,0
4,0.0,242.940,3,0.426336,0.071659,0.0,80.980,1,2021,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3289,0.0,31.920,4,0.426336,0.071659,0.0,7.980,12,2021,1,0,0,0,0,0,1,0
3290,0.0,19.440,3,0.426336,0.112887,0.0,6.480,12,2021,1,0,0,0,0,0,1,0
3291,0.0,272.940,3,0.426336,0.071659,0.0,90.980,12,2021,1,0,0,0,0,0,1,0
3292,0.2,113.568,2,0.079218,0.097119,0.0,56.784,12,2021,0,0,1,0,0,0,0,1


In [58]:
x

,discount,price,no of items,class,sub-class,Delay,price/item,month,year,location_East,location_South,location_West,segment_Corporate,segment_Home Office,delivery type_Same Day,delivery type_Second Class,delivery type_Standard Class
0,0.2,16.448,2,0.426336,0.112887,0.0,8.224,1,2018,0,0,0,0,0,0,0,1
1,0.0,29.700,5,0.426336,0.115891,0.0,5.940,1,2018,0,0,0,0,0,0,0,1
2,0.0,14.730,3,0.079218,0.045922,0.0,4.910,1,2018,0,0,0,0,0,0,0,1
3,0.0,43.920,3,0.426336,0.022046,0.0,14.640,1,2018,0,0,0,0,0,0,0,1
4,0.0,66.580,2,0.426336,0.071659,0.0,33.290,1,2018,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6695,0.0,542.940,3,0.426336,0.053464,0.0,180.980,1,2021,1,0,0,0,0,0,0,1
6696,0.2,276.784,2,0.426336,0.115891,0.0,138.392,1,2021,0,0,1,1,0,0,0,1
6697,0.0,2.040,1,0.426336,0.028773,0.0,2.040,1,2021,1,0,0,0,0,0,0,1
6698,0.1,207.846,3,0.079218,0.097119,0.0,69.282,1,2021,1,0,0,1,0,0,1,0


In [59]:
import xgboost as xgb
#xgb_clf = xgb.XGBRegressor(n_estimators=1000,learning_rate=0.2,colsample_bytree= 0.9,gamma= 0.05,max_depth= 5,reg_alpha=0.05,subsample=0.8)
xgb_clf = xgb.XGBRegressor(n_estimators=500,learning_rate=0.2,max_depth=3,gamma=0.2,subsample=0.8,reg_alpha=0.5,colsample_bytree=0.8)
xgb_clf.fit(x, y)
r2_score(y,xgb_clf.predict(x) )

0.9950797023952134

In [60]:
final=pd.DataFrame(test["id"])

final["profit"]=xgb_clf.predict(x_test)

r2_score(pd.read_csv("submission1.csv")["profit"],final["profit"])

0.9355673807308041

In [65]:
final.to_csv('submission12.csv', index=False)

In [66]:
final

,id,profit
0,6701,8.880583
1,6702,31.409077
2,6703,2.478152
3,6704,8.729888
4,6705,13.242699
...,...,...
3289,9990,-3.450006
3290,9991,9.977122
3291,9992,5.692600
3292,9993,-1.367358
